# LACNIC migration 2024-04-15

Investigate the change in VRPs during the LACNIC migration on the 15th of April

In [1]:
import io
import logging
import lzma
import tarfile

from pathlib import Path

import pandas as pd
import requests

LOG = logging.getLogger(__name__)
logging.basicConfig()
LOG.setLevel(logging.DEBUG)

from rpki_analysis.routinator import read_csv, read_csvext

In [2]:
URLS = [
    "http://josephine.sobornost.net/josephine.sobornost.net/rpkidata/2024/04/15/rpki-20240415T133558Z.tgz",
    "http://josephine.sobornost.net/josephine.sobornost.net/rpkidata/2024/04/15/rpki-20240415T110540Z.tgz"
]

import urllib.parse

csv_files = []

for url in URLS:
    tokens = urllib.parse.urlparse(url)
    file_name = tokens.path.split("/")[-1]

    p = Path(file_name)
    if not p.exists():
        LOG.info("Downloading %s", url)
        res = requests.get(url)
        with p.open("wb") as f:
            f.write(res.content)

    with tarfile.open(p, 'r:gz') as t:
        extract = [m for m in t.getmembers() if "output" in m.name]
        t.extractall(members=extract)

    for member in extract:
        if "csv" in member.name:
            csv_files.append(member.name)


In [3]:
csv_files.sort()
csv_files

['rpki-20240415T110540Z/output/rpki-client.csv',
 'rpki-20240415T133558Z/output/rpki-client.csv']

In [4]:
dfs = list(map(read_csv, csv_files))

In [5]:
before, after = dfs

In [6]:
df_merged = pd.merge(before, 
                     after,
                     how = 'outer',
                     left_on = ['asn','prefix', 'max_length'],
                     right_on = ['asn', 'prefix', 'max_length'], 
                     indicator = True)


In [7]:
disappeared = df_merged[(df_merged._merge == 'left_only') & (df_merged.trust_anchor_x == 'lacnic')]

In [8]:
disappeared

,asn,prefix,max_length,trust_anchor_x,expires_x,trust_anchor_y,expires_y,_merge
17881,270485,24.152.68.0/22,24,lacnic,1.713208e+09,NaN,NaN,left_only
43938,266086,45.4.208.0/22,23,lacnic,1.713260e+09,NaN,NaN,left_only
43952,266931,45.5.32.0/22,24,lacnic,1.713218e+09,NaN,NaN,left_only
44001,266117,45.6.12.0/22,24,lacnic,1.713251e+09,NaN,NaN,left_only
44005,0,45.6.52.0/22,32,lacnic,1.713222e+09,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...
494499,272738,2804:87e4::/32,48,lacnic,1.713228e+09,NaN,NaN,left_only
494542,273689,2804:8abc::/32,48,lacnic,1.713216e+09,NaN,NaN,left_only
494571,273765,2804:8be0::/32,48,lacnic,1.713250e+09,NaN,NaN,left_only
494572,273782,2804:8c24::/32,48,lacnic,1.713228e+09,NaN,NaN,left_only


In [9]:
res = requests.get("https://ftp.ripe.net/rpki/lacnic.tal/2024/04/15/roas.csv.xz")

with lzma.open(io.BytesIO(res.content), "rt") as f:
    df_before = read_csvext(f)

In [10]:
df_with_origin = pd.merge(
    df_merged[['asn', 'prefix', 'max_length']][(df_merged._merge == 'left_only') & (df_merged.trust_anchor_x == 'lacnic')],
    df_before,
    how = 'outer',
    left_on = ['asn','prefix', 'max_length'],
    right_on = ['asn', 'prefix', 'max_length'], 
    indicator = True)
# indicator _merge=='both' is when it disappeared in the original diff

In [11]:
display(df_with_origin[df_with_origin._merge == 'both'])
df_with_origin[df_with_origin._merge == 'both'].as_csv("20240415-lacnic-migration-delta.csv")

,asn,prefix,max_length,uri,not_before,not_after,_merge
0,270485,24.152.68.0/22,24,rsync://rpki-repo.registro.br/repo/3R6ko45HGS3...,2024-04-04 18:55:00,2025-04-03 19:00:00,both
1,266086,45.4.208.0/22,23,rsync://rpki-repo.registro.br/repo/9xvLcRDGUD2...,2024-04-15 01:31:38,2025-04-14 01:36:38,both
2,266931,45.5.32.0/22,24,rsync://rpki-repo.registro.br/repo/DAtLeGEjMG3...,2023-05-26 21:55:00,2024-05-24 22:00:00,both
3,266117,45.6.12.0/22,24,rsync://rpki-repo.registro.br/repo/HdoeVcXTsCC...,2023-07-13 14:55:00,2024-07-11 15:00:00,both
4,0,45.6.52.0/22,32,rsync://rpki-repo.registro.br/repo/A2x6icaKpVW...,2023-09-23 16:51:01,2024-09-21 16:56:01,both
...,...,...,...,...,...,...,...
1337,272738,2804:87e4::/32,48,rsync://rpki-repo.registro.br/repo/36rK8tgWRDy...,2023-09-07 17:23:00,2024-09-05 17:28:00,both
1338,273689,2804:8abc::/32,48,rsync://rpki-repo.registro.br/repo/EE8mMraphEs...,2024-03-11 19:59:27,2025-03-10 20:04:27,both
1339,273765,2804:8be0::/32,48,rsync://rpki-repo.registro.br/repo/THP8uhQACp4...,2023-12-01 18:18:52,2024-11-29 18:23:52,both
1340,273782,2804:8c24::/32,48,rsync://rpki-repo.registro.br/repo/7sWMo72DSd3...,2023-07-31 16:07:32,2024-07-29 16:12:32,both


AttributeError: 'DataFrame' object has no attribute 'as_csv'

In [ ]:
uris = set(df_with_origin[df_with_origin._merge == 'both'].uri)

In [ ]:
import urllib.parse

display(set(urllib.parse.urlparse(u).netloc for u in uris))
display([u for u in uris if 'registro.br' not in u])